## 3.1 Running Python / IPython / PySpark on the cluster

_Note:_ I haven't included the results of running cells in this notebook because I no longer have access to the test cluster on which these exercises were run.

We've created a cluster for you on AWS.

... _tour of machines on AWS and of Cloudera Manager_ ...

Everything you've done so far on your machine, you can do remotely on our cluster.

**Ex 3.1.1  [Optional, only if you use SSH / Linux regularly]  You can open an SSH connection to our cluster and run `python` and `ipython` terminals directly.  Open an SSH connection (e.g., using PuTTY) to our main machine like this:**
```
ssh client@52.29.121.97 -p 443    [port 443, which is open in your corporate firewall]
[password: client12345]
```
**Now type `python` and evaluate `2+2`.  Hit Ctrl-D to exit.  Now type `ipython` and evaluate `2+2`**

**Ex 3.1.2 [Optional, only if you use SSH / Linux regularly]  I've placed a file `pyscript.py` at in the home folder of the `client` user, with the following contents:**
```
import sys
if len(sys.argv) >= 2:
    print("Hello, {0}".format(sys.argv[1]))
else:
    print("Who are you?")
```
**Give this script a go by typing `python pyscript.py <yourname>` and `python pyscript.py`**

During all of our training, we've used IPython notebooks instead of the python terminals and python scripts.  Many people think that this is the easiest and cleanest work to run the exploratory and interactive workloads that are typical of data analytics.

Just like some of you connected to the IPython Notebook server on my laptop yesterday, you can create, edit and run code in IPython Notebooks in any other server.  I've set up an IPython Notebook on our cluster (with `pyspark` already configured) at the following address:

<center>[http://52.29.121.97](http://52.29.121.97)</center>  
<center>This server is **the most insecure server in the planet** (only half-joking here!).  **DO NOT** upload sensitive data in here.</center>

[_Note to Patrick to start up that server: In a terminal, `ssh ipython@52.29.121.97` and type `IPYTHON_OPTS=notebook pyspark`.  In another terminal, `ssh ubuntu@52.29.121.97` and type `sudo ssh ubuntu@localhost -L 0.0.0.0:80:localhost:8889`_]

The IPython Notebook is password-protected to at least hold off hackers for a few minutes.  The password is:

<center>**`client12345`**</center>

**Ex 3.1.3 Log into the IPython Notebook server on our cluster and create your own notebook.  Play around with the notebook for a few minutes to make sure everything works as you expect:**
1. **Import `numpy`, build a few arrays and perform some vector operations on them.**
2. **Import `matplotlib` and plot sin(x) from 0 to 2 $\pi$.**
3. **Import `pandas` and create a DataFrame with the GDP per capita of BE and NL.  Calculate the mean GDP per capita of BE and NL in the year 2003**
4. **Import `scikit-learn` and run the example of a linear regression on simulated data from yesterday morning**  _Note: this should also test out SciPy_

Now for something more interesting: Spark on the cluster!

**Ex 3.1.4: I've loaded all the names dataset into a single file `allnames.txt`, now with an extra year column.  The first few lines read:**
```
Mary,F,7065,1880
Anna,F,2604,1880
Emma,F,2003,1880
Elizabeth,F,1939,1880
```
**Try to read it into an RDD using `sc.textFile("file:///home/ipython/allnames.txt")`, then take the first 5 items.  What error message do you get?**

The key portion of the error message is this:
```
...node6.test-hadoop-internal.dataminded.be): java.io.FileNotFoundException: File file:/home/ipython/allnames.txt does not exist
```
The file `allnames.txt` is in the machine where we're running the driver code and IPython notebook (`manager.test-hadoop-internal.dataminded.be`), but the worker node (`node6`) is trying to read the file `allnames.txt` on its **local** filesystem.  Of course, the file _isn't_ there!

Welcome to distributed computing :-D

### HDFS

In a cluster environment, we need a **distributed file system** where:
* All the disk space on the cluster looks like one giant disk
* Any machine can access any data on the file system, regardless of where that data is located.
* Any disk failures are handled gracefully and transparently.

In a Hadoop cluster, these requirements are met by the **Hadoop Distributed File System**, or **HDFS**.

We won't talk much about the internals of HDFS, but we do want to know how to use it.  The Cloudera distribution of Hadoop includes a nice user interface to the cluster called **Hue** (Hadoop User Experience).  With it, you can explore the files in HDFS, and easily upload and download files.

Log into Hue as follows:
<center>[http://52.29.121.97:443](http://52.29.121.97:443) (MUST BE http, NOT https!)</center>

<center>Username: **`ipython`**</center>
<center>Password: **`ipython12345`**</center>

_Note to Patrick: To make this happen, edit /etc/ssh/sshd_config to remove Port 443 and restart the ssh service.  Then close the tunnel on the ubuntu machine and run the following tunnel instead: sudo ssh ubuntu@localhost -L 0.0.0.0:80:localhost:8889 -L 0.0.0.0:443:manager:8888 _

_Quick tour of Hue and of HDFS_

I've uploaded the `allnames.txt` file into HDFS, under `hdfs:///user/ipython/names/allnames.txt`.  We can use that as the parameter for `sc.textFile()`.  In fact, since the IPython notebook server is already running as the `ipython` user, if you just say this:
```
rdd = sc.textFile('names/allnames.txt')
```
then Spark already knows to look in HDFS under `/user/ipython`.

**Ex 3.1.5 Your first Spark job on the cluster!**
1. **Use Spark Core to load this file into an RDD and count the number of births between 1950 and 2000 with baby names starting in 'M'**
2. **Use Spark SQL and matplotlib to plot the number of boy and girl births for each year in the dataset**

As you're running your jobs, I'll show you the Cloudera Manager UI so that you can see the amount of CPU and disk I/O that you're exerting on the cluster.

Notice two things:
* Even though you're now running your jobs over 8 nodes with 16GB of RAM and 2 cores each, and you're reading the data from a distributed file system, the Spark code is virtually identical to what you used on your laptop.
* The overhead to distribute computation is now quite substantial.  The `allnames.txt` file is only 30MB in size.  **30MB is not Big Data**.  The overhead introduced by distributing the computation far outweighs the actual computational and network costs at this point.

**Lesson:** for anything below `100s MB to 1 GB`, use your laptop and the "small- to medium-scale" packages from early in the training (NumPy, SciPy, Pandas, Matplotlib, Scikit-learn)!  Only use the big data packages when your data actually is big.  A few exceptions:
1. Use Spark if the data is already on your cluster and you either can't move it out or it will take a long time to move it out.
2. Prepare your Spark codes on small amounts of data on your laptop to debug the easy problems.  Once your code works for small datasets, then (and only then) apply it to your Big Data.

**Ex 3.1.6 Saving your results to HDFS.  Say you want to get the complete list of unique names starting with 'M' from 1950 to 2000.  The list is quite large.**
1. **Write a Spark Core snippet to calculate that list in an RDD, but don't call `collect()` on it**
2. **Instead of using `collect()` to bring it all back to the driver node, you can use `saveAsTextFile('<yourname>.txt')` to write out the RDD to HDFS.  Do that, then use `sc.textFile()` and `take` to verify that the first few lines of the file look ok.**

**Ex 3.2.7 Go find your file in HDFS.  Does anything look odd??**

What's going on here??

<center><img src="images/saveAsTextFileMakesAFolder1.png"></center>

The "file" `patrick.txt` is actually a folder in HDFS!!  And what's in there?

<center><img src="images/saveAsTextFileMakesAFolder2.png"></center>

Multiple files??  Welcome to distributed computing :-D

What's happening is that Spark is trying to avoid sending all the data back to driver node to write it out into a single file.  Instead, Spark creates a folder called `"patrick.txt"` and **each worker node** creates a file with its share of the data.

In Hadoop clusters, this is a very common pattern used to avoid concentrating all the data in one node.  So if you ever specify a folder in the argument of `sc.textFile()`, Spark will automatically read every file in that folder and (conceptually) treat all the lines of all the files as one single RDD.

One other thing looks strange: why did Spark only use two worker nodes?

**Ex 3.1.8 When you load up `allnames.txt`, into how many partitions does Spark split the data?**

**Ex 3.1.9 You can use `repartition(N)` to force Spark to redistribute the data evenly across a different number of partitions.  With 30 MB, it makes little sense to use too many partitions.  But just for experimenting, repartition the data into 16 partitions, then run your code to count the number of births from 1950 to 2000 whose names start with 'M'.  Is is faster?  Slower?  The same?**

_Hint_: To time a piece of code, you can use the `datetime` module as follows:
```
from datetime import datetime
start = datetime.now()
... YOUR CODE HERE ...
stop = datetime.now()
elapsed_s = (stop - start).total_seconds()
print("Took {0:.2f} s".format(elapsed_s))
```

Repartitioning is usually used in two scenarios:
* You start with lots of small files (i.e., lots of small partitions) and want to coalesce the data into larger partitions to reduce the Spark overhead per partition.
* Your per-item computation is very expensive, and you want to distribute it among as many machines as possible

Repartitioning is not free: data has to be move across machines.

### Hive

Hive is a SQL-on-Hadoop server.  You can expose files in HDFS as SQL-like tables, then issue SQL-like queries against them.  This isn't a Hive training course, but I do want you to be able to access data in Hive via Spark.

_tour of Hive in Hue_

**Ex 3.1.10 Go to the Hive Editor in Hue and issue the following query to expose the names data as a table in Hive. Replace YOURNAME with your name so that we don't clash with each other:**
```
CREATE EXTERNAL TABLE hive_names_YOURNAME (
  name STRING,
  sex STRING,
  births INT,
  year INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION 'hdfs:///user/ipython/names';
```

The `CREATE TABLE` statement doesn't actually load the data.  It just tells Hive that the data exists in some location and has a CSV format with 4 columns.

**Ex 3.1.11 If you can write SQL, you can write Hive.  Run a Hive query against your table to count the number of births between 1950 and 2000 whose names start with 'M'**

A lot of your data in the production cluster is already loaded into Hive.  If you can connect Spark to Hive, you can use your production data without too much effort.  Here's how to do it:

In [ ]:
from pyspark.sql import HiveContext, Row
hiveCtx = HiveContext(sc)

tableRDD = hiveCtx.table("hive_names_patrick")  # Must run this on the cluster, not on your laptop!

That's it!  Now you can use the table from Hive like any other `SchemaRDD`.

**Ex 3.1.12 Use Spark Core to get all distinct names starting with 'M' from the year 1880 from the Hive table "hive_names_YOURNAME"**  
_WARNING_: Spark 1.3 onwards changed the type of Spark SQL objects from `SchemaRDD` to `DataFrame`, which has a different API.  To use the API that works in Spark 1.2, you have to get the `rdd` property of the table, e.g.:
```
tableRDD.rdd.filter(lambda row: row.year == 2000).count()
```

You can also just issue SQL commands that run against your Hive tables.  If you register `SchemaRDD` in the Hive context, you can even do things like join your Hive tables against temporary Spark SQL tables:

In [ ]:
hiveCtx.sql(
    "SELECT year, SUM(births) "
    "FROM hive_names_patrick GROUP BY year ORDER BY year ASC").collect()

In [ ]:
s1 = "hello\nworld"
s2 = """hello
world"""
s3 = "hello hello hello hello"
s4 = ("hello hello "
      "hello hello")

Any DDL statements that you execute will create Hive-visible tables.

**Ex 3.1.13 Using `"CREATE TABLE hive_births_YOURNAME AS SELECT ..."`, create a table in Hive with the number of births per year.  Look up this table in Hue and verify that the data is accessible from outside your Spark session now**

---

**(!) Ex 3.1.14  To get a good feel for working on the cluster and comparing cluster-code to laptop-code, upload the MLLib IPython Notebook and adapt it so that every machine learning example runs on the cluster instead of on your (or my) laptop.  You should find that there's not too much work to be done!**